# Conversion between different notations

In [2]:
import numpy as np
import math

## Tests

In [7]:
e2 = ( -np.arctan(0.25), -np.arcsin(8/9), np.arctan(4))
print(e2)
A = np.array([0.111111 , -0.888889, -0.444444, 
              0.444444 ,  0.444444, -0.777778,
              0.888889 , -0.111111,  0.444444]).reshape(3,3)

Aa = ([1/3, -2/3, 2/3], math.pi/2)
q = (1/(3 * math.sqrt(2))  , -math.sqrt(2)/3 , math.sqrt(2)/3, 1/math.sqrt(2))

(-0.24497866312686414, -1.09491407713448, 1.3258176636680326)


$ Matrix$ $notation \leftrightarrow Euler's $ $ angles $

In [5]:
def conversionFromMatrixToEulers(transformation):
    if abs(transformation[2,0]) < 1:
        # normal case
        yaw = math.atan2(transformation[1,0], transformation[0,0])
        pitch = np.arcsin(-transformation[2,0])
        rolling = math.atan2(transformation[2,1], transformation[2,2])
    elif transformation[2,0] == -1:
        # gimball lock
        yaw = math.atan2(-transformation[0,1], transformation[1,1])
        pitch = math.pi / 2
        rolling = 0
    elif transformation[2,0] == 1:
        # gimball lock
        yaw = math.atan2(-transformation[0,1], transformation[1,1])
        pitch = -math.pi / 2
        rolling = 0
    
    return (yaw, pitch, rolling)

In [6]:
conversionFromMatrixToEulers(A)

(-1.4464414707096587, 0.4605534955424451, -1.0516507663945445)

In [5]:
# normal test - without gimball lock
a = np.array([1, -8 , 4, 4, 4, -7, 8, -1, 4]).reshape(3,3)/9
a

array([[ 0.11111111, -0.88888889,  0.44444444],
       [ 0.44444444,  0.44444444, -0.77777778],
       [ 0.88888889, -0.11111111,  0.44444444]])

In [6]:
conversionFromMatrixToEulers(a)

(1.3258176636680326, -1.09491407713448, -0.24497866312686414)

In [7]:
q = 1/math.sqrt(2)
b = np.array([0,q,q,0,q,-q,-1,0,0]).reshape(3,3)
b

array([[ 0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  0.70710678, -0.70710678],
       [-1.        ,  0.        ,  0.        ]])

In [8]:
conversionFromMatrixToEulers(b)

(-0.7853981633974483, 1.5707963267948966, 0)

In [3]:
def conversionFromEulersToMatrix(EulerAngles):
    cs_yaw = math.cos(EulerAngles[0])
    sn_yaw = math.sin(EulerAngles[0])
    
    cs_pitch = math.cos(EulerAngles[1])
    sn_pitch = math.sin(EulerAngles[1])
    
    cs_rolling = math.cos(EulerAngles[2])
    sn_rolling = math.sin(EulerAngles[2])
    
    RZ = np.array([cs_yaw, -sn_yaw, 0, sn_yaw, cs_yaw, 0, 0, 0, 1]).reshape(3,3)
    RY = np.array([cs_pitch, 0, sn_pitch, 0, 1, 0, -sn_pitch, 0, cs_pitch]).reshape(3,3)
    RX = np.array([1, 0, 0, 0, cs_rolling, -sn_rolling, 0, sn_rolling, cs_rolling]).reshape(3,3)
    
    return RZ.dot( np.dot(RY, RX))

In [9]:
conversionFromEulersToMatrix(e2)

array([[ 0.44444444, -0.77777778, -0.44444444],
       [-0.11111111,  0.44444444, -0.88888889],
       [ 0.88888889,  0.44444444,  0.11111111]])

In [10]:
conversionFromEulersToMatrix((math.pi/4, 0.5, math.pi/6))

array([[ 0.62054458, -0.44286991,  0.64714038],
       [ 0.62054458,  0.78187496, -0.05996641],
       [-0.47942554,  0.43879128,  0.76000879]])

$Matrix$ $notation \leftrightarrow Angle$ $Axis$

In [15]:
def A2AngleAxis(transformation):
    eigs = np.linalg.eig(transformation)
    p = []
    
    for i in range(0,len(eigs[0])):
        # print(eigs[0][i], eigs[1][i])
        if eigs[0][i] == 1 and sum([c**2 for c in eigs[1][i]]) == 1.0:
            p = e
            break
    
    u = np.array([-p[0], p[1], 0]).reshape(3,1)
    u_prim = np.dot(transformation, u).reshape(1,3)
    angle = np.arccos(np.dot(u, u_prim))
    
    a = np.array([p[0], p[1], p[2], 
                  u[0], u[1], u[2],
                  u_prim[0], u_prim[1], u_prim[2]]).reshape(3,3)
    
    if np.linalg.det(a) < 0:
        p = (-1) * p
    
    return (p, angle)
    
    

In [16]:
A2AngleAxis(a)

(-0.11474272465971622+0.6919968721868006j) [-4.08371277e-04+0.53474409j -4.08371277e-04-0.53474409j
  5.85679803e-01+0.j        ]
(-0.11474272465971622-0.6919968721868006j) [ 0.64958832+0.j  0.64958832-0.j -0.37852063+0.j]
(1.2294854493194323+0j) [0.46679137-0.27237734j 0.46679137+0.27237734j 0.71672959+0.j        ]


IndexError: list index out of range

In [26]:
def Rodriguez(angle, vector):
    p1 = vector[0]
    p2 = vector[1]
    p3 = vector[2]
    
    q = math.sqrt(p1**2 + p2 ** 2 + p3**2)
    p1 /= q
    p2 /= q
    p3 /= q
    p = np.array([p1,p2,p3])
    pT = p.reshape(3,1)
    
    E = np.eye(3)
    px = np.array([0, -p3, p2, p3, 0, -1, -p2, p1, 0]).reshape(3,3)
    
    A = math.sin(angle) * px
    A = A + E
    A = A + (1 - math.cos(angle)) * pX.dot(px)
    
    return A
print(A)

[[ 0.111111 -0.888889 -0.444444]
 [ 0.444444  0.444444 -0.777778]
 [ 0.888889 -0.111111  0.444444]]


In [25]:
Rodriguez(vector = Aa[0], angle = Aa[1])

1.0


array([[ 1.11111111, -2.22222222, -1.33333333],
       [ 1.33333333,  1.22222222, -3.44444444],
       [ 2.22222222,  0.55555556,  1.22222222]])

In [39]:
p = np.array([1,1,1]) * (math.sqrt(3) / 3)
Rodriguez(math.pi/3, p)

array([[ 0.57735027, -0.42264973,  0.57735027],
       [ 0.57735027,  0.57735027, -0.78867513],
       [-0.42264973,  0.57735027,  0.57735027]])

$Angle$ $axis \leftrightarrow Quaternions $ 

In [21]:
def AngleAxis2Q(angle, vector):
    w = math.cos(angle/2)
    p_intensity = math.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)
    p = np.array([vector[0], vector[1], vector[2]])/p_intensity
    
    qijk = math.sin(angle/2) * p
    
    return (qijk[0], qijk[1], qijk[2], w);   

In [27]:
print(AngleAxis2Q(vector = Aa[0], angle = Aa[1]))
q

(0.2357022603955158, -0.4714045207910316, 0.4714045207910316, 0.7071067811865476)


(0.2357022603955158,
 -0.47140452079103173,
 0.47140452079103173,
 0.7071067811865475)

In [23]:
def Q2AngleAxis(q):
    i, j, k , w = q
    
    if w < 0:
        i *= -1
        j *= -1
        k *= -1
        w *= -1
    
    angle = 2 * np.arccos(w)
    
    if abs(w) == 1:
        p = (1,0,0)
    else:
        u = math.sqrt(i**2 + j**2 + k**2)
        p = (i/u, j/u, k/u)
    
    return (p,angle)
        

In [26]:
print(Q2AngleAxis(q))
Aa

((0.33333333333333326, -0.6666666666666667, 0.6666666666666667), 1.5707963267948968)


([0.3333333333333333, -0.6666666666666666, 0.6666666666666666],
 1.5707963267948966)